In [2]:
import numpy as np
import pandas as pd


In [5]:
df_train = pd.read_excel('flights.xlsx')
df = pd.read_excel('../data/test_set/flights_test.xlsx')


In [7]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [8]:
df.shape

(2671, 10)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Date_of_Journey  2671 non-null   object
 2   Source           2671 non-null   object
 3   Destination      2671 non-null   object
 4   Route            2671 non-null   object
 5   Dep_Time         2671 non-null   object
 6   Arrival_Time     2671 non-null   object
 7   Duration         2671 non-null   object
 8   Total_Stops      2671 non-null   object
 9   Additional Info  2671 non-null   object
dtypes: object(10)
memory usage: 208.8+ KB


## Convert columns type

In [10]:
###convert time to DateTime type
df['Date_of_Journey']=pd.to_datetime(df['Date_of_Journey'], format = "%d/%m/%Y")



In [11]:
# divide Dep_Time to hours and minutes 

def get_dep_hour(x):
    return x.split(sep = ':')[0]

def get_dep_min(x):
    return x.split(sep = ':')[1]

df['dep_hour'] = df.Dep_Time.apply(get_dep_hour).astype(int)
df['dep_min'] = df.Dep_Time.apply(get_dep_min).astype(int)

In [12]:
#divide years, months, days of datetime dtype
df['journey_year']=df['Date_of_Journey'].dt.year
df['journey_month']=df['Date_of_Journey'].dt.month
df['journey_day']=df['Date_of_Journey'].dt.day

In [13]:

#Add columns: arrival hours, minutes, and Arrived_next_day
def get_arr_hour(x):
    time = x.split(sep= " ")[0]
    return time.split(sep = ":")[0]
def get_arr_min(x):
    time = x.split(sep= " ")[0]
    return time.split(sep = ":")[1]


def arrived_next_day(x):
    values = x.split(sep = " ")
    return len(values) > 2



df['Arrival_hour'] = df.Arrival_Time.apply(get_arr_hour).astype(int)
df['Arrival_min'] = df.Arrival_Time.apply(get_arr_min).astype(int)
df['Arrived_next_day'] = df.Arrival_Time.apply(arrived_next_day)

In [14]:
#create columns for 1- hours, 2- mins, 3- duration converted into mins 

def get_hour(x): 
    if "h" not in x:
        return 0
    return x.split(sep = "h")[0]

def get_min(x):
    if "m" not in x: 
        return 0
    return x.split(sep = "m")[0].split()[-1]

def convert_to_mins(x):
    hours = int(get_hour(x))
    mins = int(get_min(x))
    return mins + hours * 60

df['duration_hours'] = df['Duration'].apply(get_hour)
df['duration_mins'] = df['Duration'].apply(get_min)
df['Duration_in_mins'] = df['Duration'].apply(convert_to_mins)


weekdays = Mon = 0, Tue =1, Wed = 2, Thur = 3, Fri = 4, Sat = 5, Sun = 6

In [15]:
#add weekday 
df['weekday'] = df.Date_of_Journey.dt.dayofweek

## Handling null values

In [16]:
#check for null values
df.isnull().sum()

Airline             0
Date_of_Journey     0
Source              0
Destination         0
Route               0
Dep_Time            0
Arrival_Time        0
Duration            0
Total_Stops         0
Additional Info     0
dep_hour            0
dep_min             0
journey_year        0
journey_month       0
journey_day         0
Arrival_hour        0
Arrival_min         0
Arrived_next_day    0
duration_hours      0
duration_mins       0
Duration_in_mins    0
weekday             0
dtype: int64

In [17]:
#check the instance with NaN
df1 = df[df.isna().any(axis=1)]
df1

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional Info,...,journey_year,journey_month,journey_day,Arrival_hour,Arrival_min,Arrived_next_day,duration_hours,duration_mins,Duration_in_mins,weekday


In [18]:
#drop the instance with NaN
df.dropna(inplace=True)

In [19]:
#renaming Additional Info column to be able to access .value_counts()
df.rename({'Additional Info': 'additional_info'}, axis=1, inplace=True)

In [20]:
#the feature [additional_info] contains 'No info' which counts as nulls 
df.additional_info.value_counts()


No info                         2148
In-flight meal not included      444
No check-in baggage included      76
1 Long layover                     1
Business class                     1
Change airports                    1
Name: additional_info, dtype: int64

In [21]:
#since 'No info' represent approximately 76% of the column and it appears to be missing at random, but we're waiting for more info. 
#meanwhile, we'll keep it but will not include it in the training set

## Check for duplicates

In [22]:
#check for duplicates
df.duplicated().value_counts()

False    2645
True       26
dtype: int64

In [23]:
#view duplicated rows
df[df.duplicated()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,additional_info,...,journey_year,journey_month,journey_day,Arrival_hour,Arrival_min,Arrived_next_day,duration_hours,duration_mins,Duration_in_mins,weekday
294,Jet Airways,2019-06-12,Delhi,Cochin,DEL → JAI → BOM → COK,05:30,04:25 13 Jun,22h 55m,2 stops,No info,...,2019,6,12,4,25,True,22,55,1375,2
794,Jet Airways,2019-06-12,Delhi,Cochin,DEL → JAI → BOM → COK,05:30,04:25 13 Jun,22h 55m,2 stops,In-flight meal not included,...,2019,6,12,4,25,True,22,55,1375,2
909,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,20:00,22:55,2h 55m,non-stop,No info,...,2019,3,24,22,55,False,2,55,175,6
974,Air India,2019-06-12,Kolkata,Banglore,CCU → DEL → AMD → BLR,07:00,05:25 13 Jun,22h 25m,2 stops,No info,...,2019,6,12,5,25,True,22,25,1345,2
1011,Jet Airways,2019-05-09,Delhi,Cochin,DEL → JAI → BOM → COK,05:30,04:25 10 May,22h 55m,2 stops,No info,...,2019,5,9,4,25,True,22,55,1375,3
1033,Multiple carriers,2019-05-15,Delhi,Cochin,DEL → BOM → COK,07:30,19:15,11h 45m,1 stop,No info,...,2019,5,15,19,15,False,11,45,705,2
1044,GoAir,2019-03-24,Banglore,New Delhi,BLR → DEL,20:55,23:50,2h 55m,non-stop,No info,...,2019,3,24,23,50,False,2,55,175,6
1233,Jet Airways,2019-06-06,Delhi,Cochin,DEL → JAI → BOM → COK,05:30,04:25 07 Jun,22h 55m,2 stops,No info,...,2019,6,6,4,25,True,22,55,1375,3
1504,Air India,2019-06-15,Delhi,Cochin,DEL → GOI → BOM → COK,22:00,19:15 16 Jun,21h 15m,2 stops,No info,...,2019,6,15,19,15,True,21,15,1275,5
1537,Jet Airways,2019-06-06,Delhi,Cochin,DEL → NAG → BOM → COK,06:45,04:25 07 Jun,21h 40m,2 stops,In-flight meal not included,...,2019,6,6,4,25,True,21,40,1300,3


In [24]:
#drop duplicated rows
df.drop_duplicates(keep='first',inplace=True)

## Feature Engineering

Define a function to return arrival and departure hours as parts of the day

In [25]:
#define hours of the day
def ptday(x):
    if x>=00 and x<=4:
        return 'late night'
    elif x>4 and x<=8:
        return 'early morning'
    elif x>8 and x<=12:
        return 'morning'
    elif x>12 and x<=16:
        return 'After noon'
    elif x>16 and x<=20:
        return'evening'
    elif x>20 and x<=23:
        return 'night'

In [26]:
#apply the function to arrival hours and departure hours
df['arrival_period']=df['Arrival_hour'].apply(ptday)
df['departure_period']=df['dep_hour'].apply(ptday)

## Drop redundant columns

In [27]:
#since we created new columns that extracted info from the following columns, we will drop them
df.drop(['Date_of_Journey'], axis=1, inplace=True)
df.drop(['Dep_Time'], axis=1, inplace=True)
df.drop('Arrival_Time', axis = 1, inplace = True)
# df.drop('duration_mins', axis = 1, inplace = True)
df.drop(['Duration'], axis=1, inplace=True)


#Route contains the name of cities and the number of stops. This info is already given in the Total_Stops and Destination and Source columns
#thus, we will drop Route
# df.drop(['Route'], axis=1, inplace=True)

In [28]:
df.head(10)

,Airline,Source,Destination,Route,Total_Stops,additional_info,dep_hour,dep_min,journey_year,journey_month,journey_day,Arrival_hour,Arrival_min,Arrived_next_day,duration_hours,duration_mins,Duration_in_mins,weekday,arrival_period,departure_period
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,17,30,2019,6,6,4,25,True,10,55,655,3,late night,evening
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,1 stop,No info,6,20,2019,5,12,10,20,False,4,0,240,6,morning,early morning
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,In-flight meal not included,19,15,2019,5,21,19,0,True,23,45,1425,1,evening,evening
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,8,0,2019,5,21,21,0,False,13,0,780,1,night,early morning
4,Air Asia,Banglore,Delhi,BLR → DEL,non-stop,No info,23,55,2019,6,24,2,45,True,2,50,170,0,late night,night
5,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,In-flight meal not included,18,15,2019,6,12,12,35,True,18,20,1100,2,morning,evening
6,Air India,Banglore,New Delhi,BLR → TRV → DEL,1 stop,No info,7,30,2019,3,12,22,35,False,15,5,905,1,night,early morning
7,IndiGo,Kolkata,Banglore,CCU → HYD → BLR,1 stop,No info,15,15,2019,5,1,20,30,False,5,15,315,2,evening,After noon
8,IndiGo,Kolkata,Banglore,CCU → BLR,non-stop,No info,10,10,2019,3,15,12,55,False,2,45,165,4,morning,morning
9,Jet Airways,Kolkata,Banglore,CCU → BOM → BLR,1 stop,No info,16,30,2019,5,18,22,35,False,6,5,365,5,night,After noon


## Rearrange and rename columns

**Renaming columns**

In [29]:
df.rename(columns={'Airline': "airline", 'Source':'embarked', 'Destination':'destination','Price':'price','Total_Stops':'stops'}, inplace=True)

**Rearranging columns**

In [32]:
df = df[['airline', 'embarked', 'destination', 'Route', 'stops',
       'journey_year',
       'journey_month', 'journey_day', 'weekday', 'dep_hour', 'dep_min', 'departure_period',  'Arrival_hour', 'Arrival_min', 'arrival_period',
       'Arrived_next_day',  'duration_hours', 'duration_mins',
       'Duration_in_mins' ,'additional_info']]

## Save cleaned data to new file

In [33]:
df.to_csv("./df_cleaned_test.csv", index = False)

In [34]:
cleaned = pd.read_csv("./df_cleaned_test.csv")
cleaned

,airline,embarked,destination,Route,stops,journey_year,journey_month,journey_day,weekday,dep_hour,dep_min,departure_period,Arrival_hour,Arrival_min,arrival_period,Arrived_next_day,duration_hours,duration_mins,Duration_in_mins,additional_info
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,2019,6,6,3,17,30,evening,4,25,late night,True,10,55,655,No info
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,1 stop,2019,5,12,6,6,20,early morning,10,20,morning,False,4,0,240,No info
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,2019,5,21,1,19,15,evening,19,0,evening,True,23,45,1425,In-flight meal not included
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,1 stop,2019,5,21,1,8,0,early morning,21,0,night,False,13,0,780,No info
4,Air Asia,Banglore,Delhi,BLR → DEL,non-stop,2019,6,24,0,23,55,night,2,45,late night,True,2,50,170,No info
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640,Air India,Kolkata,Banglore,CCU → DEL → BLR,1 stop,2019,6,6,3,20,30,evening,20,25,evening,True,23,55,1435,No info
2641,IndiGo,Kolkata,Banglore,CCU → BLR,non-stop,2019,3,27,2,14,20,After noon,16,55,After noon,False,2,35,155,No info
2642,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,2019,3,6,2,21,50,night,4,25,late night,True,6,35,395,No info
2643,Air India,Delhi,Cochin,DEL → BOM → COK,1 stop,2019,3,6,2,4,0,late night,19,15,evening,False,15,15,915,No info
